In [59]:
def VHI(vci,tci, scale=None):
    """ Calculate VHI (Vegetation Health Index) from time-series VCI and TCI observations. 
    
        Args:
        vci (DataArray|numpy): Time-series VCI data
        tci (DataArray|numpy): Time-series TCI data.
        scale (float|optional): a scaling factor. Default to None.
        return: 
            VHI (DataArray): Time-series VCI calculated from VIC and TCI data.
    """
    
    if not isinstance(vci, (xr.core.dataarray.DataArray, ndarray)) and not isinstance(tci, (xr.core.dataarray.DataArray, ndarray)):
        raise TypeError ("Data should be xarray DataArray or numpy")
    if not isinstance(scale, float):
        raise TypeError ("Scale should be float point")
    if vci.shape==tci.shape:
        vhi=scale*vci+(1-scale)*tci
        return vhi
    else:
        raise ValueError ("VCI and TCI have different shapes")

    
        

In [36]:
import numpy as np
import rioxarray as rx
import xarray as xr
import pfast
import datetime as dt
from dateutil.relativedelta import relativedelta

In [37]:
data=rx.open_rasterio(r"D:\Package_Development\spydis\data\MODIS_NDVI_VN.tif")
data.dims

('band', 'y', 'x')

In [51]:
test=time_dimension(data,start_date="2000-01-01", freq="day",)

In [57]:
vci=VCI(test,freq="day")

In [63]:
avi=AVI(test,freq="day")

In [65]:
type(np.array([1,2,2,2]))

numpy.ndarray

In [66]:
def TCI(data, dim=None, freq=None):
    """ Calculate TCI from time-series LST observations. 

        Args:
        data (DataArray): Time-series LST data
        dim (str|optional): A time dimension.
        frq (str|optional): A time frequency (e.g., day, month, year)
        return:
            TCI (DataArray): Time-series TCI calculated from LST data.
    """

    if not isinstance(data, xr.core.dataarray.DataArray):
        raise TypeError ("Data should be xarray DataArray")
    if dim is None:
        dim=data.dims[0].strip()
    else:
        dim=dim
    if freq is None:
        freq="month"
    else:
        freq=freq
    if freq.lower() in ["month","months","m"]:
        lst_min=data.groupby(f"{dim}.month").min(f"{dim}")
        lst_max=data.groupby(f"{dim}.month").max(f"{dim}")
        max_min=lst_max-lst_min
        nominator=data.groupby(f"{dim}.month")-lst_min
        tci=nominator.groupby(f"{dim}.month")/max_min*100
        return tci
    elif freq.lower() in ["days","day","d"]:
        lst_min=data.groupby(f"{dim}.day").min(f"{dim}")
        lst_max=data.groupby(f"{dim}.day").max(f"{dim}")
        max_min=lst_max-lst_min
        nominator=data.groupby(f"{dim}.day")-lst_min
        tci=nominator.groupby(f"{dim}.day")/max_min*100
        return tci
    elif freq.lower() in ["years","year","y"]:
        lst_min=data.min(f"{dim}")
        lst_max=data.max(f"{dim}")
        max_min=lst_max-lst_min
        nominator=data-lst_min
        tci=nominator/max_min*100
        return tci
    else:
        print("This function only supports the calculation of monthly, daily and yearly TCI.")


In [1]:
def SAVI(nir,red,scale=0.5):
    """ Calculate SAVI (Soil Adjusted Vegetation Index)
    
        Args:
            nir (numpy|DataArray): Nir infreared band
            red (numpy|DataArray): Red band in
            scale (float|optional): A scaling factor.
            
        return:
            SAVI (numpy|DataArray): A SAVI data
    """
    if isinstance(red,(xr.core.dataarray.DataArray,np.ndarray)) and isinstance(nir,(xr.core.dataarray.DataArray,np.ndarray)):
        if (red.shape==nir.shape) and (len(red.shape)==2 and len(nir.shape)==2):
            pass
        else:
            raise ValueError ("Datasets should have the same shape")
    else:
        raise TypeError("Datasets should be DataArray or numpy")
    if not isinstance(scale, float):
        raise TypeError ("scale should be float")
    savi=((nir-red)/(nir+red+scale))*(1+scale)
    return savi


In [ ]:
def NDVI(red,nir):
    """ Calculte NDVI from Red and Near Infrared bands.

        Args:
            red (numpy|DataArray): Red band in
            nir (numpy|DataArray): Nir infreared band

        return:
            NDVI (DataArray|numpy)
    """
    if isinstance(red,(xr.core.dataarray.DataArray,np.ndarray)) and isinstance(nir,(xr.core.dataarray.DataArray,np.ndarray)):
        if (red.shape==nir.shape) and (len(red.shape)==2 and len(nir.shape)==2):
            ndvi=(nir-red)/(nir+red)
            return ndvi
        else:
            raise ValueError ("Input data are not the same shape")
    else:
        raise TypeError ("Input data should be DataArray or numpy 2-D")